# Introduction to Property Packages in IDAES

Introductory stuff

Note: comment on use of methods to build up class

## Physical Parameter Block

some text

In [2]:
# Import components from Pyomo
from pyomo.environ import Constraint, Expression, Reference, Param, units as pyunits, Var
from pyomo.opt import SolverFactory

# Import IDAES cores
from idaes.core import (declare_process_block_class,
                        MaterialFlowBasis,
                        PhysicalParameterBlock,
                        StateBlockData,
                        StateBlock,
                        MaterialBalanceType,
                        EnergyBalanceType,
                        Component,
                        VaporPhase)
from idaes.core.util.initialization import (fix_state_vars,
                                            revert_state_vars,
                                            solve_indexed_blocks)
from idaes.core.util.model_statistics import degrees_of_freedom, \
                                             number_unfixed_variables
from idaes.core.util.constants import Constants as const
import idaes.logger as idaeslog

In [3]:
def define_components_and_phases(self):
    # Define Component objects for all species
    self.benzene = Component()
    self.toluene = Component()
    self.methane = Component()
    self.hydrogen = Component()
    self.diphenyl = Component()

    # Define Phase objects for all phases
    self.Vap = VaporPhase()

In [4]:
def define_basic_parameters(self):
    # Thermodynamic reference state
    self.pressure_ref = Param(mutable=True,
                              default=101325,
                              units=pyunits.Pa,
                              doc='Reference pressure')
    self.temperature_ref = Param(mutable=True,
                                default=298.15,
                                units=pyunits.K,
                                doc='Reference temperature')

    # Source: The Properties of Gases and Liquids (1987)
    # 4th edition, Chemical Engineering Series - Robert C. Reid
    self.mw_comp = Param(self.component_list,
                        mutable=False,
                        initialize={'benzene': 78.1136E-3,
                                    'toluene': 92.1405E-3,
                                    'hydrogen': 2.016e-3,
                                    'methane': 16.043e-3,
                                    'diphenyl': 154.212e-4},
                        units=pyunits.kg/pyunits.mol,
                        doc="Molecular weight")

In [5]:
def define_specific_heat_parameters(self):
    # Constants for specific heat capacity, enthalpy
    self.cp_mol_ig_comp_coeff_A = Var(
            self.component_list,
            initialize={"benzene": -3.392E1,
                        "toluene": -2.435E1,
                        "hydrogen": 2.714e1,
                        "methane": 1.925e1,
                        "diphenyl": -9.707e1},
            units=pyunits.J/pyunits.mol/pyunits.K,
            doc="Parameter A for ideal gas molar heat capacity")
    self.cp_mol_ig_comp_coeff_A.fix()

    self.cp_mol_ig_comp_coeff_B = Var(
            self.component_list,
            initialize={"benzene": 4.739E-1,
                        "toluene": 5.125E-1,
                        "hydrogen": 9.274e-3,
                        "methane": 5.213e-2,
                        "diphenyl": 1.106e0},
            units=pyunits.J/pyunits.mol/pyunits.K**2,
            doc="Parameter B for ideal gas molar heat capacity")
    self.cp_mol_ig_comp_coeff_B.fix()

    self.cp_mol_ig_comp_coeff_C = Var(
            self.component_list,
            initialize={"benzene": -3.017E-4,
                        "toluene": -2.765E-4,
                        "hydrogen": -1.381e-5,
                        "methane": -8.855e-4,
                        "diphenyl": -8.855e-4},
            units=pyunits.J/pyunits.mol/pyunits.K**3,
            doc="Parameter C for ideal gas molar heat capacity")
    self.cp_mol_ig_comp_coeff_C.fix()

    self.cp_mol_ig_comp_coeff_D = Var(
            self.component_list,
            initialize={"benzene": 7.130E-8,
                        "toluene": 4.911E-8,
                        "hydrogen": 7.645e-9,
                        "methane": -1.132e-8,
                        "diphenyl": 2.790e-7},
            units=pyunits.J/pyunits.mol/pyunits.K**4,
            doc="Parameter D for ideal gas molar heat capacity")
    self.cp_mol_ig_comp_coeff_D.fix()

    self.enth_mol_form_vap_comp_ref = Var(
            self.component_list,
            initialize={"benzene": -82.9e3,
                        "toluene": -50.1e3,
                        "hydrogen": 0,
                        "methane": -75e3,
                        "diphenyl": -180e3},
            units=pyunits.J/pyunits.mol,
            doc="Standard heat of formation at reference state")
    self.enth_mol_form_vap_comp_ref.fix()

In [6]:
@declare_process_block_class("HDAParameterBlock")
class HDAParameterData(PhysicalParameterBlock):
    CONFIG = PhysicalParameterBlock.CONFIG()

    def build(self):
        '''
        Callable method for Block construction.
        '''
        super(HDAParameterData, self).build()

        self._state_block_class = HDAStateBlock
        
        define_components_and_phases(self)
        define_basic_parameters(self)
        define_specific_heat_parameters(self)

    @classmethod
    def define_metadata(cls, obj):
        """Define properties supported and units."""
        obj.add_properties(
            {'flow_mol': {'method': None},
             'mole_frac_comp': {'method': None},
             'temperature': {'method': None},
             'pressure': {'method': None},
             'mw_comp': {'method': None},
             'dens_mol': {'method': '_dens_mol'},
             'enth_mol': {'method': '_enth_mol'}})

        obj.add_default_units({'time': pyunits.s,
                               'length': pyunits.m,
                               'mass': pyunits.kg,
                               'amount': pyunits.mol,
                               'temperature': pyunits.K})

## State Block

some text, explain two parts.

In [12]:
class _HDAStateBlock(StateBlock):

    def initialize(blk, state_args={}, state_vars_fixed=False,
                   hold_state=False, outlvl=idaeslog.NOTSET,
                   solver='ipopt', optarg=None):

        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        solve_log = idaeslog.getSolveLogger(blk.name, outlvl, tag="properties")

        # Fix state variables if not already fixed
        if state_vars_fixed is False:
            flags = fix_state_vars(blk, state_args)
        else:
            pass

        # Deactivate sum of mole fractions constraint
        for k in blk.keys():
            if blk[k].config.defined_state is False:
                blk[k].mole_fraction_constraint.deactivate()

        # Check that degrees of freedom are zero after fixing state vars
        for k in blk.keys():
            if degrees_of_freedom(blk[k]) != 0:
                raise Exception("State vars fixed but degrees of freedom "
                                "for state block is not zero during "
                                "initialization.")

        # Set solver options
        if optarg is None:
            optarg = {"tol": 1e-8}

        opt = SolverFactory('ipopt')
        opt.options = optarg

        # ---------------------------------------------------------------------
        # Initialize property calculations

        # Check that there is something to solve for
        free_vars = 0
        for k in blk.keys():
            free_vars += number_unfixed_variables(blk[k])
        if free_vars > 0:
            # If there are free variables, call the solver to initialize
            try:
                with idaeslog.solver_log(solve_log, idaeslog.DEBUG) as slc:
                    res = solve_indexed_blocks(opt, [blk], tee=slc.tee)
            except:
                res = None
        else:
            res = None

        init_log.info("Properties Initialized {}.".format(
            idaeslog.condition(res)))

        # ---------------------------------------------------------------------
        # Reactivate sum of mole fractions constraint
        for k in blk.keys():
            if blk[k].config.defined_state is False:
                blk[k].mole_fraction_constraint.activate()

        # Return state to initial conditions
        if state_vars_fixed is False:
            if hold_state is True:
                return flags
            else:
                blk.release_state(flags)

        init_log.info("Initialization Complete")

    def release_state(blk, flags, outlvl=idaeslog.NOTSET):
        '''
        Method to relase state variables fixed during initialization.
        Keyword Arguments:
            flags : dict containing information of which state variables
                    were fixed during initialization, and should now be
                    unfixed. This dict is returned by initialize if
                    hold_state=True.
            outlvl : sets output level of of logging
        '''
        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        if flags is None:
            init_log.debug("No flags passed to release_state().")
            return

        # Unfix state variables
        revert_state_vars(blk, flags)

        init_log.info_high("State Released.")

In [13]:
def add_state_variables(self):
    self.flow_mol = Var(
            initialize=1,
            bounds=(1e-8, 1000),
            units=pyunits.mol/pyunits.s,
            doc='Molar flow rate')
    self.mole_frac_comp = Var(self.component_list,
                              initialize=0.2,
                              bounds=(0, None),
                              units=pyunits.dimensionless,
                              doc="Component mole fractions")
    self.pressure = Var(initialize=101325,
                        bounds=(101325, 400000),
                        units=pyunits.Pa,
                        doc='State pressure')
    self.temperature = Var(initialize=298.15,
                           bounds=(298.15, 1500),
                           units=pyunits.K,
                           doc='State temperature')

In [14]:
def add_mole_fraction_constraint(self):
    if self.config.defined_state is False:
        self.mole_fraction_constraint = Constraint(
            expr=1e3 == sum(1e3*self.mole_frac_comp[j]
                            for j in self.component_list))

In [15]:
def add_molecular_weight_and_density(self):
    self.dens_mol = Var(initialize=1,
                        units=pyunits.m**3/pyunits.mol,
                        doc="Molar volume")

    self.ideal_gas_eq = Constraint(
        expr=self.pressure*self.dens_mol ==
        const.gas_constant*self.temperature)

In [16]:
@declare_process_block_class("HDAStateBlock",
                             block_class=_HDAStateBlock)
class HDAStateBlockData(StateBlockData):
    """
    Example property package for an ideal gas containing benzene, toluene
    hydrogen, mehtane and diphenyl.
    """

    def build(self):
        """Callable method for Block construction."""
        super(HDAStateBlockData, self).build()

        add_state_variables(self)
        add_mole_fraction_constraint(self)
        add_molecular_weight_and_density(self)
        
    def _enth_mol(self):
        # Specific enthalpy
        def enth_rule(b):
            params = self.params
            T = self.temperature
            Tr = params.temperature_ref
            return sum(self.mole_frac_comp[j] * (
                           (params.cp_mol_ig_comp_coeff_D[j]/4)*(T**4-Tr**4) +
                           (params.cp_mol_ig_comp_coeff_C[j]/3)*(T**3-Tr**3) +
                           (params.cp_mol_ig_comp_coeff_B[j]/2)*(T**2-Tr**2) +
                           params.cp_mol_ig_comp_coeff_A[j]*(T-Tr) +
                           params.enth_mol_form_vap_comp_ref[j])
                       for j in self.component_list)
        self.enth_mol = Expression(rule=enth_rule)

    def get_material_flow_terms(self, p, j):
        return self.flow_mol * self.mole_frac_comp[j]

    def get_enthalpy_flow_terms(self, p):
        """Create enthalpy flow terms."""
        return self.flow_mol * self.enth_mol

    def default_material_balance_type(self):
        return MaterialBalanceType.componentPhase

    def default_energy_balance_type(self):
        return EnergyBalanceType.enthalpyTotal

    def get_material_flow_basis(b):
        return MaterialFlowBasis.molar

    def define_state_vars(self):
        return {"flow_mol": self.flow_mol,
                "mole_frac_comp": self.mole_frac_comp,
                "temperature": self.temperature,
                "pressure": self.pressure}

## Demonstration
more text

In [17]:
from pyomo.environ import ConcreteModel, SolverFactory
from idaes.core import FlowsheetBlock

In [18]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.thermo_props = HDAParameterBlock()

m.fs.state = m.fs.thermo_props.build_state_block(
    [1], default={"defined_state": True})

In [21]:
m.fs.state.display()

print()
print("Degrees of freedom: ", degrees_of_freedom(m))

Block fs.state[1]

  Variables:
    flow_mol : Molar flow rate
        Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None : 1e-08 :     1 :  1000 : False : False :  Reals
    mole_frac_comp : Component mole fractions
        Size=5, Index=fs.thermo_props.component_list
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
         benzene :     0 :   0.2 :  None : False : False :  Reals
        diphenyl :     0 :   0.2 :  None : False : False :  Reals
        hydrogen :     0 :   0.2 :  None : False : False :  Reals
         methane :     0 :   0.2 :  None : False : False :  Reals
         toluene :     0 :   0.2 :  None : False : False :  Reals
    pressure : State pressure
        Size=1, Index=None
        Key  : Lower  : Value  : Upper  : Fixed : Stale : Domain
        None : 101325 : 101325 : 400000 : False : False :  Reals
    temperature : State temperature
        Size=1, Index=None
        Key  : Lower  : Value  : Upper : 

In [22]:
m.fs.state[1].flow_mol.fix(100)
m.fs.state[1].temperature.fix(500)
m.fs.state[1].pressure.fix(350000)
m.fs.state[1].mole_frac_comp["benzene"].fix(0.1)
m.fs.state[1].mole_frac_comp["toluene"].fix(0.4)
m.fs.state[1].mole_frac_comp["hydrogen"].fix(0.4)
m.fs.state[1].mole_frac_comp["methane"].fix(0.1)
m.fs.state[1].mole_frac_comp["diphenyl"].fix(0.0)

print("Degrees of freedom: ", degrees_of_freedom(m))

Degrees of freedom:  0


In [23]:
solver = SolverFactory('ipopt')
solver.solve(m, tee=True)

m.fs.state[1].dens_mol.display()

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

# Using the Generic Property Framework

Show how to do the same thing with the generic framework.